<a href="https://colab.research.google.com/github/EphraimElgrabli/Mixwer-ColabVersion/blob/main/Mixwer-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mixwer By Ozde - Colab Version Made By Claude.**
* 🙂 Debugged By Ephraim Elgrabli 🎩*

--------------------------------------

## **Step 1: Run this cell to install all the necassry library.**

In [ ]:
!pip install pdf2image
!pip install pillow
!pip install opencv-python
!pip install pytesseract
!pip install numpy
!pip install PyPDF4
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-heb
!wget https://github.com/tesseract-ocr/tessdata/raw/main/heb.traineddata
!mv heb.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!rm -rf Mixwer
!git clone https://github.com/EphraimElgrabli/Mixwer-ColabVersion.git
!cd Mixwer-ColabVersion/
!mkdir Images

## Step 2 - run this cell.

In [ ]:
#@title Just Click Play.
import os
from google.colab import files
from pdf2image import convert_from_path
import pytesseract
from datetime import datetime
os.chdir('/content')
os.chdir('/content/Mixwer-ColabVersion')
from Logicalscripts import *
from FunctionalScripts import *
output_directory = "/content/Mixwer-ColabVersion/Images/"
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
def upload_files():
    uploaded = files.upload()
    file_paths = list(uploaded.keys())
    return file_paths

def download_zip(zip_path):
    files.download(zip_path)

# Call the main function to handle file uploads and processing
def main():

    print("Please upload the PDF files:")
    file_paths = upload_files()

    successPdf = []
    failPdf = []

    for pdf_file_path in file_paths:
        path_original_pdf = pdf_file_path
        fileNameEnd = path_original_pdf[path_original_pdf.rfind("/") + 1:-4] + " מעורבל" + path_original_pdf[-4:]

        try:
            # Convert test .pdf to page.png
            path_originial_pages_png = functionalFiles.pdf_to_png(path_original_pdf, output_directory)

            # Merge the pages
            pathOfMerge = functionalFiles.combineFiles(path_originial_pages_png, output_directory + 'result')

            # Find how much q and a there are
            # make each q to .png
            arrayOfQuestions, numQ = exportPng.export_questions(path_originial_pages_png, output_directory)
            print("Success export Q\n")

            # make each a to .png
            for pathQ in arrayOfQuestions:
                answersId = logicalList.findNumAnswers(pathQ)
                exportPng.export_answers(pathQ, answersId, output_directory)
                print("Success export A in Q {}\n".format(pathQ))

            # crop each a
            logicalPng.cropAnswers()
            print("Success Croping\n")

            # Mix the order of the answer
            mixAnswers = logicalList.mixfiles()
            print("Success Mixing\n")

            # Make answers and questions to pages .png
            paths_of_pages = logicalPng.combineFilestoPages(mixAnswers, output_directory)
            print("Success final pages\n")

            # Add answers page
            answer_page_path, path_answers = logicalPng.createAnswersPage(mixAnswers)
            paths_of_pages = paths_of_pages + answer_page_path
            print("Success answer page\n")

            # Make .png to .pdf
            paths_of_pages_pdf = []
            for current_page in paths_of_pages:
                paths_of_pages_pdf.append(functionalFiles.png_to_pdf(current_page))
            print("Success convert pages to.pdf\n")

            ouput_pdf_path = "/content/Mixwer-ColabVersion/FinalPDFs/" + path_original_pdf[path_original_pdf.rfind("/") + 1:-4] + " מעורבל"
            functionalFiles.merge_pdf(paths_of_pages_pdf, ouput_pdf_path)
            print("Success merge pages\n")

            functionalFiles.delete_files(tuple(functionalFiles.getFilesPaths()[0] + B for B in functionalFiles.getFilesPaths()[1]))

            successPdf.append(ouput_pdf_path + ".pdf")
            print("SUCCESS {}".format(fileNameEnd))
        except Exception as e:
            print("NOT SUCCESS {}".format(fileNameEnd) + " ERROR: ", e)
            failPdf.append(pdf_file_path)

    if successPdf:
        current_datetime = datetime.now()
        datetime_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
        zip_path = f"Mixwer - {datetime_string}.zip"
        functionalFiles.zipPdf(successPdf, zip_path)
        print(f"ZIP file created: {zip_path}")
        print("Downloading the ZIP file...")
        download_zip(zip_path)
    else:
        print("No successful PDFs processed.")

    return successPdf, bool(successPdf)

if __name__ == "__main__":
    main()